# LightGBM을 이용한 categorizing 실험 (LB 0.205)

In [142]:
import numpy as np
import pandas as pd

# 0. Introduction of Data

In [143]:
products = pd.read_csv("../data/products.csv")
aisles = pd.read_csv("../data/aisles.csv")
departments = pd.read_csv("../data/departments.csv")
orders = pd.read_csv("../data/orders.csv")
prior = pd.read_csv("../data/order_products__prior.csv")
train = pd.read_csv("../data/order_products__train.csv")
frame_train = pd.read_csv("../data/frame_train.csv")
frame_test = pd.read_csv("../data/frame_test.csv")

### orders.csv

In [144]:
print("Size of orders : ", len(orders))
orders.head()

Size of orders :  3421083


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### order_products_prior.csv

In [145]:
print("Size of prior : ", len(prior))
prior.head()

Size of prior :  32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### products.csv

In [146]:
print("Size of Products : ", len(products))
products.head()

Size of Products :  49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


### frame_train.csv

In [147]:
print("Size of frame of train : ", len(frame_train))
frame_train.head()

Size of frame of train :  8474661


,user_id,order_id,product_id,products
0,1,1187899,17122,0
1,1,1187899,196,1
2,1,1187899,26405,1
3,1,1187899,46149,1
4,1,1187899,14084,0


# 1. Integrate Products Information
* products와 prior에서 얻을 수 있는 제품 관련 정보를 통합
* 기존의 attribute인 product_id, product_name, aisle_id, department_id
* 여기에 prior에서 얻을 수 있는 해당 제품에 판매에 관련된 attribute를 추가
* 총주문량(total_orders), 총재주문량(total_reorders), 재주문율(reorder_rate)을 추가함

In [148]:
productsXprior = pd.DataFrame()
productsXprior['product_id'] = prior.groupby(prior.product_id).size().index
productsXprior['product_orders'] = prior.groupby(prior.product_id).size().values
productsXprior['product_reorders'] = prior['reordered'].groupby(prior.product_id).sum().values
productsXprior['product_reorder_rate'] = (productsXprior.product_reorders / productsXprior.product_orders)
print("Size of productsXprior : ", len(productsXprior))
productsXprior.head()

Size of productsXprior :  49677


,product_id,product_orders,product_reorders,product_reorder_rate
0,1,1852,1136,0.613391
1,2,90,12,0.133333
2,3,277,203,0.732852
3,4,329,147,0.446809
4,5,15,9,0.600000


In [149]:
Products = products.merge(productsXprior, on='product_id', how='outer')
print("Size of Products : ", len(Products))
Products.head()

Size of Products :  49688


,product_id,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
0,1,Chocolate Sandwich Cookies,61,19,1852.0,1136.0,0.613391
1,2,All-Seasons Salt,104,13,90.0,12.0,0.133333
2,3,Robust Golden Unsweetened Oolong Tea,94,7,277.0,203.0,0.732852
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,329.0,147.0,0.446809
4,5,Green Chile Anytime Sauce,5,13,15.0,9.0,0.600000


In [150]:
Products[Products.isnull().any(axis=1)]

,product_id,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
3629,3630,Protein Granola Apple Crisp,57,14,NaN,NaN,NaN
3717,3718,Wasabi Cheddar Spreadable Cheese,21,16,NaN,NaN,NaN
7044,7045,Unpeeled Apricot Halves in Heavy Syrup,88,13,NaN,NaN,NaN
25382,25383,Chocolate Go Bites,61,19,NaN,NaN,NaN
27498,27499,Non-Dairy Coconut Seven Layer Bar,100,21,NaN,NaN,NaN
36232,36233,Water With Electrolytes,100,21,NaN,NaN,NaN
37702,37703,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...,75,17,NaN,NaN,NaN
43724,43725,Sweetart Jelly Beans,100,21,NaN,NaN,NaN
45970,45971,12 Inch Taper Candle White,101,17,NaN,NaN,NaN
46624,46625,Single Barrel Kentucky Straight Bourbon Whiskey,31,7,NaN,NaN,NaN


In [151]:
Products = Products.replace(np.NaN, 0)
print("Size of Products : ", len(Products))
Products.head()

Size of Products :  49688


,product_id,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
0,1,Chocolate Sandwich Cookies,61,19,1852.0,1136.0,0.613391
1,2,All-Seasons Salt,104,13,90.0,12.0,0.133333
2,3,Robust Golden Unsweetened Oolong Tea,94,7,277.0,203.0,0.732852
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,329.0,147.0,0.446809
4,5,Green Chile Anytime Sauce,5,13,15.0,9.0,0.600000



# categorized_order_hour_of_day

In [152]:
orders['categorized_order_hour_of_day'] = 0
orders.loc[orders['order_hour_of_day']>=6,'categorized_order_hour_of_day'] = 1
orders.loc[orders['order_hour_of_day']>=12,'categorized_order_hour_of_day'] = 2
orders.loc[orders['order_hour_of_day']>=18,'categorized_order_hour_of_day'] = 3
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,categorized_order_hour_of_day
0,2539329,1,prior,1,2,8,NaN,1
1,2398795,1,prior,2,3,7,15.0,1
2,473747,1,prior,3,3,12,21.0,2
3,2254736,1,prior,4,4,7,29.0,1
4,431534,1,prior,5,4,15,28.0,2


# 2. Integrate_Users_Information
* prior와 orders를 통합(priorXorders)하여 user_id, order_id, product_id를 하나로 모음.
* priorXorders를 기반으로 하여 유저 정보를 담고 있는 Users를 생성.

In [153]:
priorXorders = prior.merge(orders, on='order_id')
print("Size of Prior : ", len(priorXorders))
priorXorders.head()

Size of Prior :  32434489


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,categorized_order_hour_of_day
0,2,33120,1,1,202279,prior,3,5,9,8.0,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,1


In [154]:
user_orders = pd.DataFrame()
user_orders['user_avg_orders_period'] = orders.groupby('user_id')['days_since_prior_order'].mean()
user_orders['user_total_orders'] = orders.groupby('user_id').size()

user_orders['user_id'] = user_orders.index
user_orders.index.name = None

print("Size of user_orders : ", len(user_orders))
user_orders.head()

Size of user_orders :  206209


,user_avg_orders_period,user_total_orders,user_id
1,19.000000,11,1
2,16.285714,15,2
3,12.000000,13,3
4,17.000000,6,4
5,11.500000,5,5


In [155]:
user_prior = pd.DataFrame()
user_prior['user_total_products'] = priorXorders.groupby('user_id').size()
user_prior['ordered_products_set'] = priorXorders.groupby('user_id')['product_id'].apply(set)
user_prior['user_unique_products'] = (user_prior.ordered_products_set.map(len))
user_prior = user_prior[['user_total_products', 'user_unique_products', 'ordered_products_set']]

user_prior['user_id'] = user_prior.index
user_prior.index.name = None

print("Size of user_prior : ", len(user_prior))
user_prior.head()

Size of user_prior :  206209


,user_total_products,user_unique_products,ordered_products_set,user_id
1,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",1
2,195,102,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",2
3,88,33,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",3
4,18,17,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",4
5,37,23,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",5


In [156]:
Users = user_prior.merge(user_orders, on='user_id')
print("Size of users : ", len(Users))
Users.head()

Size of users :  206209


,user_total_products,user_unique_products,ordered_products_set,user_id,user_avg_orders_period,user_total_orders
0,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",1,19.000000,11
1,195,102,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",2,16.285714,15
2,88,33,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",3,12.000000,13
3,18,17,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",4,17.000000,6
4,37,23,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",5,11.500000,5


In [157]:
Users['user_avg_cart'] = (Users.user_total_products / Users.user_total_orders)
print("Size of users : ", len(Users))
Users.head()

Size of users :  206209


,user_total_products,user_unique_products,ordered_products_set,user_id,user_avg_orders_period,user_total_orders,user_avg_cart
0,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",1,19.000000,11,5.363636
1,195,102,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",2,16.285714,15,13.000000
2,88,33,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",3,12.000000,13,6.769231
3,18,17,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",4,17.000000,6,3.000000
4,37,23,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",5,11.500000,5,7.400000


# 3. Train Data Generation
* ### 미리 만들어 둔 frame_train을 기반으로 위의 orders, Products, Users 정보를 통합시켜 train data를 만듬.
* order_prior, order_train은 학습데이터로 포함 X, order_test가 없기 때문에 test데이터에 해당 attribute를 포함시킬 수 없음.

In [158]:
train_df = frame_train.copy()
train_df.set_index(['user_id','order_id','product_id']).head(10)

products
user_id order_id product_id          
1       1187899  17122              0
                 196                1
                 26405              1
                 46149              1
                 14084              0
                 13032              1
                 26088              1
                 39657              1
                 12427              0
                 25133              1

In [159]:
train_df = train_df.merge(Products, how='left', on='product_id')
train_df.head()

,user_id,order_id,product_id,products,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
0,1,1187899,17122,0,Honeycrisp Apples,24,4,13880.0,9377.0,0.675576
1,1,1187899,196,1,Soda,77,7,35791.0,27791.0,0.776480
2,1,1187899,26405,1,XL Pick-A-Size Paper Towel Rolls,54,17,1214.0,536.0,0.441516
3,1,1187899,46149,1,Zero Calorie Cola,77,7,8558.0,6953.0,0.812456
4,1,1187899,14084,0,Organic Unsweetened Vanilla Almond Milk,91,16,15935.0,12923.0,0.810982


In [160]:
train_df = train_df.merge(Users, how='left', on='user_id')
train_df.head()

,user_id,order_id,product_id,products,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate,user_total_products,user_unique_products,ordered_products_set,user_avg_orders_period,user_total_orders,user_avg_cart
0,1,1187899,17122,0,Honeycrisp Apples,24,4,13880.0,9377.0,0.675576,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636
1,1,1187899,196,1,Soda,77,7,35791.0,27791.0,0.776480,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636
2,1,1187899,26405,1,XL Pick-A-Size Paper Towel Rolls,54,17,1214.0,536.0,0.441516,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636
3,1,1187899,46149,1,Zero Calorie Cola,77,7,8558.0,6953.0,0.812456,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636
4,1,1187899,14084,0,Organic Unsweetened Vanilla Almond Milk,91,16,15935.0,12923.0,0.810982,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636


In [161]:
train_df = train_df.merge(orders, how='left', on=['user_id','order_id'])
train_df.head()

,user_id,order_id,product_id,products,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate,...,ordered_products_set,user_avg_orders_period,user_total_orders,user_avg_cart,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,categorized_order_hour_of_day
0,1,1187899,17122,0,Honeycrisp Apples,24,4,13880.0,9377.0,0.675576,...,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636,train,11,4,8,14.0,1
1,1,1187899,196,1,Soda,77,7,35791.0,27791.0,0.776480,...,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636,train,11,4,8,14.0,1
2,1,1187899,26405,1,XL Pick-A-Size Paper Towel Rolls,54,17,1214.0,536.0,0.441516,...,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636,train,11,4,8,14.0,1
3,1,1187899,46149,1,Zero Calorie Cola,77,7,8558.0,6953.0,0.812456,...,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636,train,11,4,8,14.0,1
4,1,1187899,14084,0,Organic Unsweetened Vanilla Almond Milk,91,16,15935.0,12923.0,0.810982,...,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,11,5.363636,train,11,4,8,14.0,1


### 모든 데이터 통합 완료 & 사용할 attribute 선정

In [162]:
train_df.columns

Index(['user_id', 'order_id', 'product_id', 'products', 'product_name',
       'aisle_id', 'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'user_total_products', 'user_unique_products',
       'ordered_products_set', 'user_avg_orders_period', 'user_total_orders',
       'user_avg_cart', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order',
       'categorized_order_hour_of_day'],
      dtype='object')

In [163]:
features = [
    # product information
    'aisle_id', 'department_id', 'product_orders', 'product_reorders', 'product_reorder_rate', 
    # user information
    'user_total_products', 'user_unique_products', 'user_avg_orders_period', 'user_total_orders', 'user_avg_cart', 
    # order information
    'order_number', 'order_dow', 'categorized_order_hour_of_day', 'days_since_prior_order']

In [164]:
train_X = train_df[features]
train_y = train_df['products']
train_X

,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate,user_total_products,user_unique_products,user_avg_orders_period,user_total_orders,user_avg_cart,order_number,order_dow,categorized_order_hour_of_day,days_since_prior_order
0,24,4,13880.0,9377.0,0.675576,59,18,19.000000,11,5.363636,11,4,1,14.0
1,77,7,35791.0,27791.0,0.776480,59,18,19.000000,11,5.363636,11,4,1,14.0
2,54,17,1214.0,536.0,0.441516,59,18,19.000000,11,5.363636,11,4,1,14.0
3,77,7,8558.0,6953.0,0.812456,59,18,19.000000,11,5.363636,11,4,1,14.0
4,91,16,15935.0,12923.0,0.810982,59,18,19.000000,11,5.363636,11,4,1,14.0
5,121,14,3751.0,2465.0,0.657158,59,18,19.000000,11,5.363636,11,4,1,14.0
6,23,19,2523.0,1360.0,0.539041,59,18,19.000000,11,5.363636,11,4,1,14.0
7,45,19,5019.0,3846.0,0.766288,59,18,19.000000,11,5.363636,11,4,1,14.0
8,23,19,6476.0,4797.0,0.740735,59,18,19.000000,11,5.363636,11,4,1,14.0
9,21,16,6196.0,4586.0,0.740155,59,18,19.000000,11,5.363636,11,4,1,14.0


# 4. Test Data Generation

In [165]:
test_df = frame_test.copy()
test_df = test_df.merge(Products, how='left', on='product_id')
test_df = test_df.merge(Users, how='left', on='user_id')
test_df = test_df.merge(orders, how='left', on=['user_id','order_id'])

In [166]:
test_X = test_df[features]

# 5. Learning & Prediction

In [167]:
import lightgbm as lgb

In [168]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 100

### Learning

In [169]:
train_X_ds = lgb.Dataset(train_X, label=train_y, categorical_feature=['aisle_id','department_id'])

In [170]:
bst = lgb.train(params, train_X_ds, ROUNDS)

### Prediction

In [171]:
result = bst.predict(test_X)

In [172]:
result = pd.concat([frame_test,pd.DataFrame(result, columns=['predict'])], axis=1)

In [173]:
result.head()

,user_id,order_id,product_id,predict
0,3,2774568,17668,0.180718
1,3,2774568,44683,0.134829
2,3,2774568,48523,0.104866
3,3,2774568,21903,0.278211
4,3,2774568,14992,0.153332


# 6. Prediction Probability 기반 최종 products list 생성

In [174]:
TRESHOLD = 0.22

d = dict()
for row in result.itertuples():
    if row.predict > TRESHOLD:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in orders[orders.eval_set == 'test'].order_id:
    if order not in d:
        d[order] = 'None'

In [175]:
output = pd.DataFrame.from_dict(d, orient='index')
output.reset_index(inplace=True)
output.columns = ['order_id', 'products']
output.head()

,order_id,products
0,2774568,21903 21137 22035 47766 42265 16797 28373 23650
1,1528013,21903
2,1376945,33572 44632 14947 12384 13176
3,1356845,11520 13176
4,2161313,11266 196 10441 43721 12427 37710 48142


In [176]:
output.to_csv('categorized_hour-lightGBM.csv', index=False)